In [ ]:
import os
from PIL import Image

import pandas as pd
import numpy as np
from tqdm import tqdm

from torchvision import transforms, models
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torch import nn, optim

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('breast-cancer.csv')

In [ ]:
df.info()

In [ ]:
df = df.drop(columns=['id'], errors='ignore')

df['diagnosis'] = df['diagnosis'].map({'B': 0, 'M': 1})

X = df.drop(columns=['diagnosis']).values
y = df['diagnosis'].values

In [ ]:
activation_dict = {
    "relu": nn.ReLU,
    "tanh": nn.Tanh,
    "sigmoid": nn.Sigmoid
}

In [ ]:
def create_model(input_dim, hidden_config, output_dim=1):
    """
    Создаёт MLP-модель с заданной конфигурацией:
      hidden_config: [(n_neurons, activation_name), (n_neurons, activation_name), ...]
      output_dim: количество нейронов на выходном слое (у нас 1, т.к. бинарная классификация)
    """
    layers = []
    in_dim = input_dim

    for (n_neurons, act_name) in hidden_config:
        layers.append(nn.Linear(in_dim, n_neurons))
        layers.append(activation_dict[act_name]())
        in_dim = n_neurons

    # Выходной слой
    layers.append(nn.Linear(in_dim, output_dim))
    # Для бинарной классификации (через BCEWithLogitsLoss) активацию Sigmoid не добавляем здесь
    model = nn.Sequential(*layers)
    return model

In [ ]:
def train_model(model, 
                X_train, y_train, 
                X_val, y_val, 
                epochs=20, 
                batch_size=32, 
                lr=1e-3, 
                device='cpu',
                optimizer='Adam'):
    X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
    X_val_t   = torch.tensor(X_val,   dtype=torch.float32).to(device)
    y_val_t   = torch.tensor(y_val,   dtype=torch.float32).view(-1, 1).to(device)
    
    model.to(device)

    if optimizer == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer == 'NAG':
        optimizer = optim.SGD(model.parameters(), lr=lr, nesterov=True, momentum=0.9)
    elif optimizer == 'RMSProp':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    elif optimizer == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=lr)
    elif optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr)
    else:
        raise ValueError('Unknow optimizer parameter')
        
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        model.train()
        permutation = torch.randperm(X_train_t.size(0))
        for i in range(0, X_train_t.size(0), batch_size):
            optimizer.zero_grad()
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X_train_t[indices], y_train_t[indices]

            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_t)
        val_preds = (torch.sigmoid(val_outputs) >= 0.5).float()
        correct = (val_preds == y_val_t).sum().item()
        val_acc = correct / len(y_val_t)

    return val_acc

In [ ]:
def evaluate_model_cv(hidden_config, X, y, n_splits=5, epochs=20, batch_size=32, lr=1e-3, device='cpu', optimizer='Adam', verbose=False):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    fold_accuracies = []
    
    iterator = enumerate(skf.split(X, y))
    
    if verbose:
        iterator = tqdm(iterator)

    for idx, (train_idx, val_idx) in iterator:
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val   = scaler.transform(X_val)

        input_dim = X.shape[1]
        model = create_model(input_dim, hidden_config)

        val_acc = train_model(model, 
                              X_train, y_train, 
                              X_val,   y_val, 
                              epochs=epochs, 
                              batch_size=batch_size, 
                              lr=lr, 
                              device=device,
                              optimizer=optimizer)
        fold_accuracies.append(val_acc)

    return np.mean(fold_accuracies)

In [ ]:
evaluate_model_cv(
    [(16, "relu")],
    X,
    y,
    lr=1e-2,
    n_splits=100,
    device='cuda',
    optimizer='SGD',
    verbose=True,
)

In [ ]:
evaluate_model_cv(
    [(16, "relu")],
    X,
    y,
    lr=1e-2,
    n_splits=100,
    device='cuda',
    optimizer='NAG',
    verbose=True,
)

In [ ]:
evaluate_model_cv(
    [(16, "relu")],
    X,
    y,
    lr=1e-2,
    n_splits=100,
    device='cuda',
    optimizer='Adagrad',
    verbose=True,
)

In [ ]:
evaluate_model_cv(
    [(16, "relu")],
    X,
    y,
    lr=1e-2,
    n_splits=100,
    device='cuda',
    optimizer='RMSProp',
    verbose=True,
)

In [ ]:
evaluate_model_cv(
    [(16, "relu")],
    X,
    y,
    lr=1e-2,
    n_splits=100,
    device='cuda',
    optimizer='Adam',
)

# Genetic algorithm

In [ ]:
from deap import base, creator, tools, algorithms

In [ ]:
"""
Будем кодировать гиперпараметры так:
- Число скрытых слоёв H: 1..3 (например)
- Для каждого слоя: количество нейронов N: 4..64 (пример диапазона)
- Для каждого слоя: функция активации: ['relu', 'tanh', 'sigmoid']

То есть, если H=2, то особь должна хранить:
[
  (n_neurons_layer1, activation_layer1),
  (n_neurons_layer2, activation_layer2)
]

Если H=1 — только один слой, если H=3 — три слоя и т.д.

Для упрощения можно хранить в особи структуру вида:
[H, n1, act1, n2, act2, n3, act3]

Но придётся аккуратно интерпретировать в функции evaluate.
"""

H_min, H_max = 1, 5
N_min, N_max = 4, 128
activations = list(activation_dict.keys())  # ["relu", "tanh", "sigmoid"]

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

In [ ]:
toolbox.register("num_layers", np.random.randint, H_min, H_max+1)
toolbox.register("n_neurons", np.random.randint, N_min, N_max+1)
toolbox.register("act", lambda: np.random.choice(activations))

In [ ]:
def init_ind():
    # Индивид = [H, n1, a1, n2, a2, n3, a3]
    return [
        toolbox.num_layers(),
        toolbox.n_neurons(), toolbox.act(),
        toolbox.n_neurons(), toolbox.act(),
        toolbox.n_neurons(), toolbox.act(),
        toolbox.n_neurons(), toolbox.act(),
        toolbox.n_neurons(), toolbox.act(),
    ]

In [ ]:
toolbox.register("individual", tools.initIterate, creator.Individual, init_ind)

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
def decode_individual(ind):
    H = ind[0]
    hidden_config = []
    for i in range(H):
        n_i = ind[1 + i*2]
        a_i = ind[2 + i*2]
        hidden_config.append((n_i, a_i))
    return hidden_config

In [ ]:
def eval_individual(ind):
    hidden_config = decode_individual(ind)

    acc = evaluate_model_cv(hidden_config, X, y, 
                            n_splits=100,
                            epochs=20, 
                            batch_size=32,
                            lr=1e-2,
                            device='cuda')
    return (acc,)

In [ ]:
toolbox.register("evaluate", eval_individual)

In [ ]:
def cx_individual(ind1, ind2):
    cxpoint = np.random.randint(1, len(ind1))
    ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]
    return ind1, ind2

In [ ]:
def mut_individual(ind, indpb=0.1):
    for i in range(len(ind)):
        if np.random.rand() < indpb:
            if i == 0:
                ind[i] = np.random.randint(H_min, H_max+1)
            elif i % 2 == 1:
                ind[i] = np.random.randint(N_min, N_max+1)
            else:
                if i > 0:
                    ind[i] = np.random.choice(activations)
    return (ind,)

In [ ]:
toolbox.register("mate", cx_individual)
toolbox.register("mutate", mut_individual, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
def run_ga(n_generations=5, population_size=10):
    pop = toolbox.population(n=population_size)
    hof = tools.HallOfFame(1)  # сохраняем лучшее решение
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("mean", np.mean)
    stats.register("max", np.max)

    pop, logbook = algorithms.eaSimple(pop, toolbox, 
                                       cxpb=0.7,  # вероятность скрещивания
                                       mutpb=0.3,  # вероятность мутации
                                       ngen=n_generations, 
                                       stats=stats, 
                                       halloffame=hof, 
                                       verbose=True)
    
    print("Лучшее решение:", hof[0])
    print("Fitness (accuracy) этого решения:", hof[0].fitness.values[0])
    
    best_hidden_config = decode_individual(hof[0])
    print("Декодированная конфигурация (слои):", best_hidden_config)

    return pop, hof, logbook

In [ ]:
import time

start = time.monotonic()
final_pop, hall_of_fame, logs = run_ga(n_generations=10, population_size=15)
end = time.monotonic()

In [ ]:
(end - start) / 60

In [ ]:
with open('output.txt', 'w') as f:
    f.write(str(hall_of_fame))
    f.write('\n\n\n')
    f.write(str(logs))

In [ ]:
hall_of_fame[0]

In [ ]:
decode_individual(hall_of_fame[0])